# Automated ML
## Introduction

This notebook is automatically generated by the Fabric low-code AutoML wizard based on your selections. Whether you're building a regression model, a classifier, or another machine-learning solution, this tool simplifies the process by transforming your goals into executable code. You can easily modify any settings or code snippets to better align with your requirements.

### What is FLAML?

[FLAML (Fast and Lightweight Automated Machine Learning)](https://aka.ms/fabric-automl) is an open-source AutoML library designed to quickly and efficiently find the best machine learning models and hyperparameters. FLAML optimizes for speed, accuracy, and cost, making it an excellent choice for a wide range of machine-learning tasks.

### Steps in this notebook

1. **Load the data**: Import your dataset.
2. **Generate features**: Automatically transform and preprocess your data to improve model performance.
3. **Use AutoML to find your best model**: Use FLAML to automatically select the most suitable model and optimize its parameters.
4. **Save the final machine learning model**: Store the trained model for future use.
5. **Generate predictions**: Use the saved model to predict outcomes on new data.

> [!IMPORTANT]
> **The forecasting functionality is currently supported only on Pandas DataFrames.**
> **Automated ML is currently supported on Fabric Runtimes 1.2+ or any Fabric environment with Spark 3.4+.**


In [ ]:
%pip install scikit-learn==1.5.1


StatementMeta(, 1731a218-ab15-4617-a532-15744a738792, 19, Finished, Available, Finished)


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Default notebook optimization

This cell configures the logging and warning settings to reduce unnecessary output and focus on critical information. It suppresses specific warnings and logs from the underlying libraries, ensuring a cleaner and more readable notebook experience.

In [ ]:
import logging
import warnings
 
logging.getLogger('synapse.ml').setLevel(logging.CRITICAL)
logging.getLogger('mlflow.utils').setLevel(logging.CRITICAL)
warnings.simplefilter('ignore', category=FutureWarning)
warnings.simplefilter('ignore', category=UserWarning)

StatementMeta(, 1731a218-ab15-4617-a532-15744a738792, 21, Finished, Available, Finished)

## Step 1: Load the Data

This cell is responsible for importing the raw data from the specified source into the notebook environment. The data could come from various sources, such as a file or table in your lakehouse.

Once loaded, this data will serve as the input for subsequent steps, such as data transformation, model training, and evaluation.

In [ ]:
import re
import pandas as pd
import numpy as np


df = spark.read.option("header", "true").option("inferSchema", "true").csv(
    "Files/forecasting/modelGeneratedData/overall_monthly_with_economic_and_future.csv"
).cache()
# Transform to pandas according to the selected models
X = df.limit(100000).toPandas() # Use df.toPandas() to use all the data
X = X.rename(columns = lambda c:re.sub('[^A-Za-z0-9_]+', '_', c))  # Replace not supported characters in column name with underscore to avoid invalid character for model training and saving

target_col = re.sub('[^A-Za-z0-9_]+', '_', "Sales_USD") 

StatementMeta(, 1731a218-ab15-4617-a532-15744a738792, 27, Finished, Available, Finished)

In [ ]:
display(X)
print(f"Target column: {target_col}")
print(f"Available columns: {X.columns.tolist()}")

StatementMeta(, 1731a218-ab15-4617-a532-15744a738792, 28, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, dd766f7f-fa94-49cb-966e-0ee5c84b4861)

Target column: Sales_USD
Available columns: ['Date', 'Quantity_Invoiced', 'Quantity_Invoiced_Mean', 'Quantity_Invoiced_Count', 'Sales_USD', 'Sales_USD_Mean', 'Sales_USD_Count', 'Unique_Customers', 'Unique_Products', 'Unique_Categories', 'Unique_SubCategories', 'Unique_EndMarkets_L1', 'Unique_EndMarkets_L2', 'future_orders_count', 'future_orders_qty_total', 'future_orders_qty_next_1m', 'future_orders_qty_next_3m', 'future_orders_qty_next_6m', 'future_orders_qty_next_12m', 'future_orders_sales_next_1m', 'future_orders_sales_next_3m', 'future_orders_sales_next_6m', 'future_orders_sales_next_12m', 'future_orders_avg_lead_time', 'future_orders_min_lead_time', 'future_orders_max_lead_time', 'future_orders_due_next_month', 'future_orders_due_next_quarter', 'future_orders_unique_customers', 'future_orders_unique_products', 'future_to_current_ratio', 'qty_rolling_avg_3m', 'future_to_rolling_3m_ratio', 'qty_rolling_avg_12m', 'future_to_rolling_12m_ratio', 'has_future_orders', 'high_future_orders

## Step 2: Generate features

Featurization is the process of transforming raw data into a format optimized for training a machine learning model. It ensures the model can access the most relevant information, significantly impacting its accuracy and performance.

This step applies various techniques to refine the data, enhance its quality, and make it compatible with the selected algorithms, helping the model learn patterns more effectively.

In [ ]:
# Set Functions if needed for Featurization
def create_fillna_processor(
    df, mean_features=None, median_features=None, mode_features=None
):
    """
    Create a ColumnTransformer that fills missing values in a DataFrame using different strategies
    based on the skewness of the numerical features and the specified feature lists.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    mean_features (list, optional): List of features to impute using the mean strategy. Defaults to None.
    median_features (list, optional): List of features to impute using the median strategy. Defaults to None.
    mode_features (list, optional): List of features to impute using the mode strategy. Defaults to None.

    Returns:
    ColumnTransformer: A fitted ColumnTransformer that can be used to transform the DataFrame.
    list: List of all features supported by SimpleImputer in the DataFrame.
    list: List of datetime features in the DataFrame.
    """
    if mean_features is None:
        mean_features = []
    if median_features is None:
        median_features = []
    if mode_features is None:
        mode_features = []
    all_features = mean_features + median_features + mode_features
    # Group features by their imputation needs
    mean_features = [
        col
        for col in df.select_dtypes(include=["number"]).columns
        if df[col].skew(skipna=True) <= 1 and col not in all_features
    ] + mean_features
    median_features = [
        col
        for col in df.select_dtypes(include=["number"]).columns
        if df[col].skew(skipna=True) > 1 and col not in all_features
    ] + median_features
    all_features = mean_features + median_features
    datetime_features = df.select_dtypes(include=["datetime"]).columns.tolist()
    mode_features = [col for col in df.columns.tolist() if col not in all_features + datetime_features]

    transformers = []

    if mean_features:
        transformers.append(
            ("mean_imputer", SimpleImputer(strategy="mean"), mean_features)
        )
    if median_features:
        transformers.append(
            ("median_imputer", SimpleImputer(strategy="median"), median_features)
        )
    if mode_features:
        transformers.append(
            ("mode_imputer", SimpleImputer(strategy="most_frequent"), mode_features)
        )

    column_transformer = ColumnTransformer(transformers=transformers)
    all_features = mean_features + median_features + mode_features

    return column_transformer.fit(df), all_features, datetime_features


def fillna(df, processor, all_features, datetime_features):
    """
    Fill missing values in a DataFrame using a specified processor and mode imputation.

    Parameters:
    df (pd.DataFrame): The input DataFrame with missing values.
    processor (object): An object with a `transform` method that processes the DataFrame.
    all_features (list): List of all features supported by SimpleImputer in the DataFrame.
    datetime_features (list): List of datetime features in the DataFrame.

    Returns:
    pd.DataFrame: A DataFrame with missing values filled.
    """
    filled_array = processor.transform(df)
    filled_df = pd.DataFrame(filled_array, columns=all_features)
    if datetime_features:
        datetime_data = df[datetime_features]
        datetime_data.ffill()
        filled_df = pd.concat([datetime_data, filled_df], axis=1)
    for col in df.columns:
        filled_df[col].fillna(filled_df[col].mode()[0], inplace=True)

    return filled_df


StatementMeta(, 1731a218-ab15-4617-a532-15744a738792, 29, Finished, Available, Finished)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer


time_col = "Date"
ts_col = X.pop(time_col)
X.insert(0, time_col, ts_col.apply(lambda x: np.datetime64(x, "ns")))

# convert object type to nearest dtype
X = X.convert_dtypes()
X = X.dropna(axis=1, how='all')

# select columns for model training
X = X.select_dtypes(include=['number', 'datetime', 'category'])

from sklearn.model_selection import train_test_split

# You may need to update the test_size based on your scenario
X_train, X_test = train_test_split(X, test_size=int(X.shape[0] / 12 * 0.2) * 12, shuffle=False, random_state=41)

mean_features, median_features, mode_features = [], [], []
 
preprocessor, all_features, datetime_features = create_fillna_processor(X_train, mean_features, median_features, mode_features)
X_train = fillna(X_train, preprocessor, all_features, datetime_features)
X_test = fillna(X_test, preprocessor, all_features, datetime_features)
 
y_train = X_train.pop(target_col)
y_test = X_test.pop(target_col)

display(X_train[:10])


StatementMeta(, 1731a218-ab15-4617-a532-15744a738792, 30, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 977e1e09-625d-4bf4-829f-107701ea86a3)

## Step 3: Use AutoML to find your best model

We will now use FLAML's AutoML to automatically find the best machine learning model for our data. AutoML (Automated Machine Learning) simplifies the model selection process by automatically testing and tuning various algorithms and configurations, helping us quickly identify the most effective model with minimal manual effort.

### Tracking results with experiments in Fabric

Experiments in Fabric let you track the results of your AutoML process, providing a comprehensive view of all the metrics and parameters from your trials.

In [ ]:
# MLFlow Logging Related

import mlflow

mlflow.autolog(exclusive=False)
mlflow.set_experiment("Forecast_Dollar_Prediction_v1_da")


StatementMeta(, 1731a218-ab15-4617-a532-15744a738792, 31, Finished, Available, Finished)

2025/07/31 19:09:44 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


2025/07/31 19:09:45 INFO mlflow.tracking.fluent: Experiment with name 'Forecast_Dollar_Prediction_v1_da' does not exist. Creating a new experiment.


<Experiment: artifact_location='', creation_time=1753988986854, experiment_id='575fadb5-35ad-4f84-8425-c9a5c38c04db', last_update_time=None, lifecycle_stage='active', name='Forecast_Dollar_Prediction_v1_da', tags={}>

#### Configure the AutoML trial and settings

These configurations are driven by the AutoML mode and task selected in the wizard. For example, if you select "quick prototype", you'll see a setting for time budget.

In [ ]:
# Import the AutoML class from the FLAML package
import flaml
from flaml import AutoML

# Define AutoML settings
settings = {
    "time_budget": 3600, # Total running time in seconds
    "estimator_list": [
        "arima",
        "avg",
        "extra_tree",
        "holt-winters",
        "lassolars",
        "lgbm",
        "naive",
        "prophet",
        "rf",
        "savg",
        "snaive",
        "sarimax",
        "tcn",
        "xgboost",
        "xgb_limitdepth"
    ],
    "task": "ts_forecast",  # Task type 
    "log_file_name": "flaml_experiment_dollar.log",  # FLAML log file
    "seed": 41 , # Random seed 
    "mlflow_exp_name": "Forecast_Dollar_Prediction_v1_da",  # MLflow experiment name
    "use_spark": True, # whether to use Spark for distributed training
    "n_concurrent_trials": 3,  # the maximum number of concurrent trials 
    "verbose": 1, 
    "featurization": "auto",
}

if flaml.__version__ > "2.3.3":
    settings["entrypoint"] = "low-code"

# Create an AutoML instance
automl = AutoML(**settings)


StatementMeta(, 1731a218-ab15-4617-a532-15744a738792, 32, Finished, Available, Finished)

2025/07/31 19:10:02 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.ml.


2025/07/31 19:10:02 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


2025/07/31 19:10:04 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.


2025/07/31 19:10:15 INFO mlflow.tracking.fluent: Autologging successfully enabled for transformers.
2025/07/31 19:10:17 INFO mlflow.tracking.fluent: Autologging successfully enabled for pytorch_lightning.


#### Run the AutoML trial

Run the AutoML trial, with all trials being tracked as experiment runs. The trial is performed on the processed dataset, using the `Exited` variable as the target, and applying the defined configurations for optimal model selection.

In [ ]:
with mlflow.start_run(nested=True, run_name="Forecast_Dollar_Prediction_v1_da"):
    automl.fit(
        X_train=X_train, 
        y_train=y_train,  # target column of the training data 
        period=12, 
    )

StatementMeta(, 1731a218-ab15-4617-a532-15744a738792, 34, Finished, Available, Finished)

INFO:flaml.automl.task.time_series_task:Couldn't import orbit, skipping
[I 2025-07-31 19:27:03,746] A new study created in memory with name: optuna


[I 2025-07-31 19:27:15,687] A new study created in memory with name: optuna


## Step 4: Save the final machine learning model

Upon completing the AutoML trial, you can now save the final, tuned model as an ML model in Fabric.

In [ ]:
model_path = f"runs:/{automl.best_run_id}/model"

# Register the model to the MLflow registry
registered_model = mlflow.register_model(model_uri=model_path, name="Forecast_Dollar_Prediction_v1_da")

# Print the registered model's name and version
print(f"Model '{registered_model.name}' version {registered_model.version} registered successfully.")

StatementMeta(, 1731a218-ab15-4617-a532-15744a738792, 35, Finished, Available, Finished)

Successfully registered model 'Forecast_Dollar_Prediction_v1_da'.


Created version '1' of model 'Forecast_Dollar_Prediction_v1_da'.


## Step 5: Generate predictions

1. Generate predictions.

In [ ]:
loaded_model_pred = automl.predict(X_test)
print('Predicted dollar values', loaded_model_pred)


StatementMeta(, 1731a218-ab15-4617-a532-15744a738792, 36, Finished, Available, Finished)

Predicted dollar values 98     1.490795e+06
99     1.564212e+06
100    1.621928e+06
101    1.738255e+06
102    1.564272e+06
103    1.705338e+06
104    1.791218e+06
105    1.482651e+06
106    1.505137e+06
107    1.523865e+06
108    1.784614e+06
109    1.780827e+06
110    1.262489e+06
111    1.583074e+06
112    1.503310e+06
113    1.578833e+06
114    1.592718e+06
115    1.780646e+06
116    1.843816e+06
117    1.562336e+06
118    1.844597e+06
119    1.682779e+06
120    1.974714e+06
121    1.912190e+06
0      1.500737e+06
1      1.554110e+06
2      1.505587e+06
3      1.585905e+06
4      1.588146e+06
5      1.815314e+06
6      1.844928e+06
7      1.603866e+06
8      1.890144e+06
9      1.651526e+06
10     2.009489e+06
11     1.897667e+06
12     1.763315e+06
13     1.637706e+06
14     1.560424e+06
15     1.487153e+06
16     1.609024e+06
17     1.572489e+06
18     1.924610e+06
19     1.572210e+06
20     1.827815e+06
21     1.576816e+06
22     1.844941e+06
23     1.741811e+06
Name: Sales_USD,

2. Save the predictions to a table.

In [ ]:
from pyspark.sql.types import FloatType
predictions = spark.createDataFrame(loaded_model_pred, FloatType())
saved_name = "dollar_prediction_v1_da_predictions".replace(".", "_")
predictions.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save(f"Tables/{saved_name}")

StatementMeta(, 1731a218-ab15-4617-a532-15744a738792, 37, Finished, Available, Finished)

In [ ]:
from sklearn.metrics import r2_score
import pandas as pd
import re
'''
y_true = spark.read.option("header", "true").csv(
    "Files/forecasting/modelGeneratedData/overall_monthly_with_economic_and_future.csv"
).toPandas()[target_col].tail(48).reset_index(drop=True)
y_true = spark.read.option("header", "true").option("inferSchema", "true").csv(
    "Files/forecasting/modelGeneratedData/overall_monthly_with_economic_and_future.csv"
).toPandas()["Sales_USD"].tail(48).reset_index(drop=True)
'''
y_pred = spark.read.table("dollar_prediction_v1_da_predictions") \
    .toPandas()["value"].reset_index(drop=True)

original_df = spark.read.option("header", "true").option("inferSchema", "true").csv(
    "Files/forecasting/modelGeneratedData/overall_monthly_with_economic_and_future.csv"
).toPandas()

original_df = original_df.rename(columns = lambda c:re.sub('[^A-Za-z0-9_]+', '_', c))

y_true = original_df["Sales_USD"].tail(48).reset_index(drop=True)

print(f"R² Score for best dollar prediction model: {r2_score(y_true, y_pred):.4f}")

StatementMeta(, 93c665cf-e58a-45b1-b99c-177958d5cf7d, 6, Finished, Available, Finished)

R² Score for best dollar prediction model: -0.5373
